# Annex

This notebook gather the mathematical proofs and in-depth explanations of the functioning of neural networks. It includes the mathematics of forward and back propagation and visualizations.  
This content is not necessary to use neural networks as popular libraries like TensorFlow and PyTorch embed all these computations. However, it is helpful for those who seek a deeper understanding of what happens under the hood in Deep Learning

## Basics
  
### Linear algebra
  
Let's start with some vocabulary and concepts:
- scalars are elements of $\mathbb{R}$
- vectors are elements of $\mathbb{R}^n$
- matrix are elements of $\mathbb{R}^{n \times m}$  

Matrices are two dimensional. Sometimes we need more dimensions, for example RGB images live in $\mathbb{R}^{n \times m \times 3}$, and batches of RGB images live in $\mathbb{R}^{b \times n \times m \times 3}$. Hence we need to generalize matrices to higher dimensions. This generalization is called a **tensor**. The tensor product is the generalization of the matrix product. For a 3d tensor $T$ we need 3 indices:
$$T_{i,j,k} = x$$  

### Chain rule

The basics of Deep Learning is to optimize a function with respect to a model parameters. Let's say a neural network models a function $F$ and has 3 layers of parameters $P_1$, $P_2$ and $P_3$ and we want to minimize a loss $L$. The groundtruth is $Y$ and the data is $X$. Then the training is:
$$min_{P_1,P_2,P_3} L(F(X), Y)$$
We want to compute:
$$\frac{\partial L}{\partial P_i},\quad \forall i \in \{1, 2, 3\}$$
There is no closed form for these derivatives (except very special cases), hence neural networks use gradient descent algorithms. Also, it is not possible (again except very special cases) to consider all the dataset at once, hence neural networks use batch training. The required hypothesis is that the samples are iid in a batch, this is why you need to randomly shuffle your dataset so the statistical properties of a batch are as close as possible to those of the whole dataset.  
Now let's consider that $F(X) = P_3(P_2(P_1(X)))$, this is what happens when you stack layers in a neural network. The chain rule states that:
$$\frac{\partial L}{\partial P1} = \frac{\partial L}{\partial P_3}\frac{\partial P_3}{\partial P_2}\frac{\partial P_2}{\partial P_1}$$  
This means we can use the gradients of a layer to compute the gradients of the previous layer. The gradients are propagated backward, hence the name **backward propagation**.

### Matrix differentiation

Let's consider a function:
$$f: \mathbb{R}^n \longrightarrow \mathbb{R}^m, \quad x \longrightarrow [f_1(x_1,...,x_n),...,f_m(x_1,...,x_n)]$$
We could compute the derivative of this function with respect to each element, except that it gets quite slow. We would rather compute it in matrix form. The basic building bloc is the **Jacobian matrix**:
$$J_f(x) = \frac{\partial f}{\partial x} = \begin{bmatrix} \frac{\partial f_1}{\partial x_1} & ... & \frac{\partial f_1}{\partial x_n}
\\ ... & ... & ...
\\ \frac{\partial f_m}{\partial x_1} & ... & \frac{\partial f_m}{\partial x_n} \end{bmatrix}$$
The dimension of the Jacobian matrix is *output dimension* x *input dimension*, so in this case it is $\mathbb{R}^{m \times n}$  
The chain rule in Deep Learning is equivalent to multiplying the Jacobian matrices. For a function $g: x \longrightarrow f(x)$, the derivative with respect to $x$ is:
$$\frac{\partial g}{\partial x} = \frac{\partial g}{\partial f} \frac{\partial f}{\partial x}$$
The Jacobians are not necessarily matrices, actually most of the time they are **tensors**. The backward propagation consists in computing a flow of Jacobians, that is the reason behind the name of the popular library **TensorFlow**.

## Linear layer

### Forward propagation
  
The forward propagation of a linear layer implements the following function for a vector input $x$:
$$f(x) = Wx + b$$

### Backward propagation
  
For the backward propagation we need to compute $\frac{\partial L}{\partial W}$ and $\frac{\partial L}{\partial b}$ where $L$ is the loss that we are trying to minimize. According to the chain rule:
$$\frac{\partial L}{\partial W} = \frac{\partial L}{\partial f} \frac{\partial f}{\partial W}$$
$$\frac{\partial L}{\partial b} = \frac{\partial L}{\partial f} \frac{\partial f}{\partial b}$$
We get $\frac{\partial L}{\partial f}$ from the next layer.  
First, we are going to do what is called a pro-gamer move. We add a one to $x$ and a column to $W$ which is $b$, and call the outputs $x^*$ and $W^*$. So:
$$f: \mathbb{R}^{n+1 \times m} \longrightarrow \mathbb{m}, \quad x \longrightarrow Wx + b = W^* x^*$$
Then the Jacobian is a third order tensor in $\mathbb{R}^{n+1 \times m \times m}$. Let's compute the derivative element by element. First, let's consider $f_i: x \longrightarrow \sum_j w_{ij}x_j$ an element of the matrix $f(x)$, and a weight $w_{kl}$, and compute one element of the Jacobian tensor:
$$\left( \frac{\partial f}{\partial W} \right)_{ikl} = \frac{\partial f_i}{\partial w_{kl}} = \frac{\partial}{\partial w_{kl}} \left( \sum_j w_{ij} x_j \right)$$
$$= \sum_j \delta_{ik} \delta_{jl} x_j$$
$$= \delta_{ik} x_l$$  
This tensor would look like this:  
![jacobianW](images/jacobian_W.png)  
Then $\frac{\partial L}{\partial W}$ is the tensor product where each element is:
$$ \left( \frac{\partial L}{\partial W} \right)_{ijkl} = \sum_i \frac{\partial L}{\partial f_i} \delta_{ik} x_l =  \frac{\partial L}{\partial f_i} x_l$$  
This can be represented as a matrix ! Actually the derivative is the outter product between the propagated gradients and $x$:
$$ \frac{\partial L}{\partial W} = \left( \frac{\partial L}{\partial f} \right)^T x$$
And we deduce that:
$$ \frac{\partial L}{\partial b} = \frac{\partial L}{\partial f} $$

## Convolutional layer

### Forward propagation
  
The input of a convolutional layer is a matrix $X$ (or tensor) and the output is a matrix $O$ (or tensor). The parameter is a kernel $W$ usually small. For a 2x2 kernel the first element of the output is:
$$o_{11} = w_{11} x_{11} + w_{12} x_{12} + w_{21} x_{21} + w_{22} x_{22}$$  
Generally,
$$o_{ij} = \sum_{kl} w_{kl} x_{i-1+k, j-1+l}$$
This can be computed as an inner product between a vectorized $W \in \mathbb{R}^{k \times l}$ and the vectorized patch of $x \in \mathbb{R}^{k \times l}$ (same size of the kernel):
$$o_{ij} = vec(w)^T vec(x_{k \in [i, i+k], l \in [j, j+l]})$$
This trick is well known and implemented in the algorithm *im2col*.  

### Backward propagation

The derivative $\frac{\partial O}{\partial W}$ is a tensor of order 4 (trust me on that). This is hard to compute and to visualize. We use a trick to compute it. Notice that:
$$\frac{\partial o_{ij}}{\partial w_{lk}} = x_{i-1+l,j-1+k} = \left( \frac{\partial O}{\partial W} \right)_{i,j,k,l}$$  
Now bear with me. What we are really interested in is the update of each weight $w_{ij}$ of the kernel, for that we need to compute the derivative $\frac{\partial L}{\partial W}$. Using the chain rule:
$$\frac{\partial L}{\partial W} = \frac{\partial L}{\partial O}\frac{\partial O}{\partial W}$$
$\frac{\partial L}{\partial O}$ are the gradients that we get from the next layer and if we decompose by each element:
$$\frac{\partial L}{\partial w_{lk}} = \sum_{ij} \frac{\partial L}{\partial o_{ij}} \frac{\partial o_{ij}}{\partial w_{lk}} = \sum_{ij} \frac{\partial L}{\partial o_{ij}} x_{i-1+l,j-1+k}$$
Let's write this in an example:
$$\frac{\partial L}{\partial w_{11}} = 
\frac{\partial L}{\partial o_{11}} x_{11} + 
\frac{\partial L}{\partial o_{12}} x_{12} + 
\frac{\partial L}{\partial o_{21}} x_{21} + 
\frac{\partial L}{\partial o_{22}} x_{22}$$
That looks awfully similar to this convolution:   
![2d conv_input](images/backward_conv.png) 